<a href="https://colab.research.google.com/github/Corosso/ValoAI/blob/main/Project_ValoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn import svm

# Get the data 
acs = pd.read_csv("teams.csv")
map_win_rate = pd.read_csv("map_win_rate.csv")
previous_matches = pd.read_csv("previous_matches.csv")


In [2]:
# Split the ACS values into separate columns
acs_s = acs['acs'].str.split(',', expand=True).astype(float)
acs_s.columns = ['acs_1', 'acs_2', 'acs_3', 'acs_4', 'acs_5']

# Concatenate the team name with the ACS values
acs_merged = pd.concat([acs['team'], acs_s], axis=1)

# Merge ACS and map win rate
acs_merged = pd.merge(acs_merged, map_win_rate, on="team")

In [3]:
# Remove the number in parentheses from the map name
map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')

<ipython-input-3-1b7a5e3ef39c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')


In [4]:
#Convert team names to numeric values using label encoder
encoder_teams = LabelEncoder()
acs_merged['team'] = encoder_teams.fit_transform(acs_merged['team'])



In [5]:
# Convert team_A and team_B names to numeric values using label encoder
previous_matches['team_A'] = encoder_teams.transform(previous_matches['team_A'])
previous_matches['team_B'] = encoder_teams.transform(previous_matches['team_B'])

In [6]:
# Merge previous matches with acs and win rate
data = pd.merge(previous_matches, acs_merged, left_on="team_A", right_on='team',how='outer')
data = pd.merge(previous_matches, acs_merged, left_on="team_B", right_on='team',how='inner')

In [7]:
# Replace ("-") for NaN
data['win_rate'] = data['win_rate'].replace('-', float('nan'))
# Replace NaN for zeros
data = data.fillna(0)
# Convert map names to numeric values using label encoder
encoder_maps = LabelEncoder()
data['map'] = encoder_maps.fit_transform(data['map'])

# Convert win rate to float
data['win_rate'] = data['win_rate'].str.rstrip('%').astype(float) / 100


In [14]:
# Replace NaN for zeros
data = data.fillna(0)

In [15]:

# Select the input variables (ACS and Map Win Rate)
cols = [ 'acs_1', 'acs_2', 'acs_3', 'acs_4', 'acs_5', 'map', 'win_rate',"team_A","team_B"]
X = data[cols]

# Select the output variable (winner)
y = data["winner"]

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7 )

# Create a logistic regression model
model = LogisticRegression(max_iter=10000)
model2 = svm.SVC(kernel="rbf",C=0.01, gamma="auto")

# Train the model with the training data
model.fit(X_train, y_train)
model2.fit(X_train, y_train)

y_predict=model.predict(X_test)
y_predict2=model2.predict(X_test)

# Evaluate model accuracy with test data
precision = f1_score(y_test, y_predict)
precision2 = f1_score(y_test, y_predict2)

print("Model precission: ", precision)
print("Model 2 precission: ", precision2)

Model precission:  0.8715083798882682
Model 2 precission:  0.736


In [ ]:
data

In [ ]:
# Teams table with its label or tag
teams = pd.read_csv("teams.csv")
teams.drop('acs',inplace=True, axis=1)
teams_encoded = encoder_teams.transform(teams['team'])
teams_encoded = pd.DataFrame(teams_encoded, columns=['Tag'])
teams_table=pd.merge(teams, teams_encoded,left_index=True, right_index=True)
teams_table

In [85]:
# Make predictions on new data
# Which teams are going to play
team_A=teams_table.loc[7,'Tag']
team_B=teams_table.loc[4,'Tag']
t={'teams':[team_A, team_B]}
team = pd.DataFrame(data=t)

data_teams = pd.merge(X,team, right_on="teams", left_on="team_A", how='outer')
data_teams = pd.merge(X,team, right_on="teams", left_on="team_B",how='outer')



In [84]:
team

,teams
0,6
1,8


In [86]:
data_teams

,acs_1,acs_2,acs_3,acs_4,acs_5,map,win_rate,team_A,team_B,teams
0,233.4,194.8,203.4,189.0,163.0,25.0,0.74,8.0,7.0,NaN
1,233.4,194.8,203.4,189.0,163.0,55.0,0.55,8.0,7.0,NaN
2,233.4,194.8,203.4,189.0,163.0,100.0,0.63,8.0,7.0,NaN
3,233.4,194.8,203.4,189.0,163.0,11.0,0.72,8.0,7.0,NaN
4,233.4,194.8,203.4,189.0,163.0,72.0,0.73,8.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...
221,212.3,195.0,219.3,156.3,137.7,38.0,0.25,11.0,13.0,NaN
222,212.3,195.0,219.3,156.3,137.7,42.0,0.50,11.0,13.0,NaN
223,212.3,195.0,219.3,156.3,137.7,84.0,0.00,11.0,13.0,NaN
224,212.3,195.0,219.3,156.3,137.7,80.0,1.00,11.0,13.0,NaN


In [ ]:
# Get the probabilities
X_teams = data_teams[cols]
probs = model.predict_proba(X_teams)

# Get the winner
#if probs[0][1] > probs[0][0]:
#    winner = team_B
#    prob_winner = probs[0][1]
#else:
#    winner = team_A
#    prob_winner = probs[0][0]

probs
# Print probs and winner
#print("Winner is:", winner)
#print("Probs for: ",team_A, ":", probs[0][0])
#print("Probs for:", team_B, ":", probs[0][1])
#print("Winner probs:", prob_winner)

In [27]:
teams_table

,team,Tag
0,Team Heretics,17
1,Karmine Corp,10
2,FNATIC,4
3,BBL Esports,1
4,KOI,8
5,Team Vitality,19
6,Team Liquid,18
7,FUT Esports,6
8,Natus Vincere,15
9,Giants Gaming,7
